This notebook generates abstracts with our selected and finetuned models togetehr with recursion to generate abstracts using the whole text without being limited to token number.

** All the datasets, generated summaries, and evaluation results can be found in my shared Google Drive folder: https://drive.google.com/drive/folders/1sNoJxaShjifrt_AqyG5_sZYGxHknqfOM?usp=sharing

In [ ]:
from google.colab import drive
from huggingface_hub import notebook_login

drive.mount('/content/drive')

notebook_login()
# https://huggingface.co/settings/tokens

model_index = "bart-pubmed-20k"
my_model = "Chung-Fan/bart-pubmed-20k"
og_model = "facebook/bart-large-cnn"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Mon Apr 29 17:24:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0              45W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install ipython-autotime
%load_ext autotime

time: 258 µs (started: 2024-04-29 17:24:16 +00:00)


In [ ]:
# important variables
my_num_beams = 3
max_token_length = 1024 # max token for recursive chunking (will experiment with less chunks)
portion_summary = int(max_token_length/3)
summary_max_length = portion_summary # make sure recursive chunks are large enough for summary
recurisve_batch_size = 1
overlap_percent = 20
print_var = False

time: 540 µs (started: 2024-04-29 17:24:16 +00:00)


In [ ]:
!pip install datasets py7zr rouge_score bert_score

time: 5.63 s (started: 2024-04-29 17:24:16 +00:00)


In [ ]:
import datasets
from datasets import load_metric, Dataset, DatasetDict
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import transformers
import pandas as pd
import itertools

scaledown = 20000
summary_max_length = 400
my_num_beams = 3 # 8 to 3, to make it less computational intensive

rouge_metric = load_metric("rouge")
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
bertscore_metric = load_metric("bertscore")
bertscore_names = ["precision", "recall", "f1"]
#
metrics = [rouge_metric, bertscore_metric]
metric_names = [str(rouge_metric.name), str(bertscore_metric.name)]

# model_index = "bart-pubmed-20k"
# my_model = "Chung-Fan/bart-pubmed-20k"
# og_model = "facebook/bart-large-cnn"

# model_index = "distilbart-pubmed-20k"
# my_model = "Chung-Fan/distilbart-pubmed-20k"
# og_model = "philschmid/distilbart-cnn-12-6-samsum"

# model_index = "pegasus-pubmed-20k"
# my_model = "Chung-Fan/pegasus-pubmed-20k"
# og_model = "tuner007/pegasus_summarizer"

# # tokenzier model: use a model that takes in 4k+ tokens to tokenize before chunking for recursion
# tokenizer_index = "longformer-pubmed-20k"
# my_tokenizer = "Chung-Fan/longformer-pubmed-20k"
# long_tokenizer_model = "hyesunyun/update-summarization-bart-large-longformer"



<ipython-input-6-34a0a7a454d2>:14: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


time: 8.19 s (started: 2024-04-29 17:24:22 +00:00)


# Load processed dataset

In [ ]:
# Load the CSV file into a DataFrame
token_df = pd.read_csv('/content/drive/My Drive/pubmed/token_df.csv')
token_df_test = pd.read_csv('/content/drive/My Drive/pubmed/token_df_test.csv')
token_df_val = pd.read_csv('/content/drive/My Drive/pubmed/token_df_val.csv')

# token1024_df = pd.read_csv('/content/drive/My Drive/pubmed/token1024_df.csv')
# token1024_df_test = pd.read_csv('/content/drive/My Drive/pubmed/token1024_df_test.csv')
# token1024_df_val = pd.read_csv('/content/drive/My Drive/pubmed/token1024_df_val.csv')

time: 12.6 s (started: 2024-04-29 17:24:30 +00:00)


In [ ]:
# dataset
# Convert DataFrames to Hugging Face Datasets
dataset_train = Dataset.from_pandas(token_df)
dataset_test = Dataset.from_pandas(token_df_test)
dataset_val = Dataset.from_pandas(token_df_val)

# Create DatasetDict
dataset_dict = DatasetDict({
    'train': dataset_train,
    'test': dataset_test[:100],
    'validation': dataset_val
})

dataset_med = dataset_dict
dataset_med


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'article_len', 'abstract_len', 'article', 'abstract'],
        num_rows: 10700
    })
    test: {'Unnamed: 0': [0, 1, 3, 6, 7, 10, 11, 15, 16, 17, 20, 21, 22, 24, 27, 28, 30, 32, 33, 34, 35, 37, 38, 39, 40, 42, 47, 49, 50, 53, 54, 56, 57, 58, 59, 63, 70, 71, 73, 78, 80, 81, 82, 83, 84, 86, 91, 92, 95, 96, 101, 103, 104, 107, 110, 112, 114, 116, 118, 119, 122, 123, 124, 125, 126, 127, 130, 131, 132, 134, 135, 137, 138, 139, 140, 141, 144, 145, 147, 148, 151, 153, 156, 157, 158, 160, 161, 163, 166, 170, 171, 172, 173, 174, 175, 176, 177, 178, 180, 181], 'article_len': [3820, 3409, 1170, 2273, 3444, 2547, 3883, 676, 1268, 1406, 4078, 2396, 4081, 2280, 3186, 1609, 1857, 2185, 2541, 1532, 1669, 3083, 821, 2027, 2687, 3785, 2260, 1635, 1652, 1934, 2039, 950, 1080, 4085, 1497, 2746, 1714, 1867, 2713, 1663, 2195, 1911, 3937, 2452, 3166, 3187, 1683, 2022, 1765, 2180, 1334, 1910, 1206, 3144, 1357, 1508, 2843, 1618, 2675, 3691, 2

time: 1.04 s (started: 2024-04-29 17:24:43 +00:00)


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

time: 46.7 ms (started: 2024-04-29 17:24:44 +00:00)


# Start recursive process

In [ ]:

tokenizer = AutoTokenizer.from_pretrained(my_model)
long_tokenizer = AutoTokenizer.from_pretrained(long_tokenizer_model)

model = AutoModelForSeq2SeqLM.from_pretrained(my_model).to(device)

tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/45.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/827 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

time: 13.5 s (started: 2024-04-29 17:26:10 +00:00)


# Recusively generate summary for bigger input (doc of 4096 tokens summarized by models with input size of 1024 tokens)

In [ ]:
# rerun
print_var = False
def summarize(text, maxSummarylength=summary_max_length): # layer 2
    # Encode the text and summarize
    inputs = tokenizer.encode("summarize: " +
                              text,
                              return_tensors="pt",
                              max_length=1024, truncation=True)

    summary_ids = model.generate(inputs.to(device), max_length=maxSummarylength,
                                 min_length=int(maxSummarylength/5),
                                 length_penalty=10.0,
                                 num_beams=my_num_beams, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

def split_text_into_pieces(text,
                           max_tokens=1024,
                           overlapPercent=20): # layer 2
    # Tokenize the text
    tokens = long_tokenizer.tokenize(text)
    if print_var:
      print('tokens: ', tokens)
    # Calculate the overlap in tokens
    overlap_tokens = int(max_tokens * overlapPercent / 100)

    # Split the tokens into chunks of size
    # max_tokens with overlap
    pieces = [tokens[i:i + max_tokens]
              for i in range(0, len(tokens),
                             max_tokens - overlap_tokens)]

    # Convert the token pieces back into text
    text_pieces = [long_tokenizer.decode(
        long_tokenizer.convert_tokens_to_ids(piece),
        skip_special_tokens=True) for piece in pieces]
    if print_var:
      print('text_pieces: ', text_pieces)
    return text_pieces

def recursive_summarize(text, max_length=max_token_length, recursionLevel=0, overlapPercent=overlap_percent):
# layer 1: final_summary = recursive_summarize(text)
    recursionLevel=recursionLevel+1

    tokens = long_tokenizer.tokenize(text)

    if print_var:
      print('text: ', text)
      print("######### Recursion level: ", recursionLevel,"\n\n######### ")
      print('len(tokens): ', len(tokens),'max_length: ', max_length )

    expectedCountOfChunks = len(tokens)/max_length
    max_length=int(len(tokens)/expectedCountOfChunks)+2

    # Break the text into pieces of max_length
    pieces = split_text_into_pieces(text, max_tokens=max_length, overlapPercent=overlapPercent)
    if print_var:
      print('pieces: ', pieces)
    summaries=[]
    k=0

    for k in range(0, len(pieces)):
        piece=pieces[k]
        summary =summarize(piece, maxSummarylength=int(max_length/3*2))
        summaries.append(summary)
        if print_var:
          print("****************************************************")
          print("Piece:",(k+1)," out of ", len(pieces), "pieces")
          print(piece, "\n")
          print("SUMNMARY: ", summary)
          print("****************************************************")

    concatenated_summary = ' '.join(summaries)

    # If the summary at this recursive level is still too long, repeat the process
    tokens = tokenizer.tokenize(concatenated_summary)

    # print('len(tokens):', len(tokens), ', max len:', max_length)
    if len(tokens) > max_length:

        # print("############# GOING RECURSIVE ##############")
        return recursive_summarize(concatenated_summary,
                                   max_length=max_length,
                                   recursionLevel=recursionLevel)
    else:

      # Concatenate the summaries and summarize again
        final_summary=concatenated_summary
        if len(pieces)>1:
            final_summary = summarize(concatenated_summary,
                                  maxSummarylength=max_length)

        # print('final_summary: ', final_summary)
        return final_summary

# rerun
def chunks(list_of_elements, batch_size):
    """Yield successive batch-sized chunks from list_of_elements."""

    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def evaluate_summaries(dataset, metrics, model, tokenizer, batch_size=recurisve_batch_size, device=device,
                                   column_text="article",
                                   column_summary="abstract"):
    '''Calculate respective rouge metric for the given data'''
    article_batches = list(chunks(dataset[column_text], batch_size)) # article batches
    abstract_batches = list(chunks(dataset[column_summary], batch_size))  # abstract batches

    decoded_summaries_lst = []

    i = 0
    for article_batch, abstract_batch in tqdm(zip(article_batches, abstract_batches), total=len(article_batches)):
      decoded_summaries = [recursive_summarize(article_batch[0])] # 0 because batch is 1

      print('i: ', i)
      if i == 0:
        print('-------- \n decoded summary quality check: ', decoded_summaries, '\n')
      i += 1
      # store generated summary
      decoded_summaries_lst.append(decoded_summaries)
      if print_var:
        print('debug1, len(decoded_summaries): ', len(decoded_summaries), '\n --- \n decoded_summaries: ', decoded_summaries, )
        print('abstract baatch: ', len(abstract_batch), '\n --- \n abstract_batch:', abstract_batch)

      for metric in metrics:
        metric.add_batch(predictions=decoded_summaries, references=abstract_batch)  # add this batch to the metric

    scores = dict()
    for metric in metrics:
      if str(metric.name) == 'bert_score':
        scores['bert_score'] = metric.compute(lang="en") # Calculate final metric score
      else:
        scores[str(metric.name)] = metric.compute() # Calculate final metric score

    return scores, decoded_summaries_lst


time: 2.55 ms (started: 2024-04-29 17:26:24 +00:00)


In [ ]:
# # # rerun
# model_index = "pegasus-pubmed-20k"
# my_model = "Chung-Fan/pegasus-pubmed-20k"
# og_model = "tuner007/pegasus_summarizer"

# tokenizer = AutoTokenizer.from_pretrained(my_model)
# long_tokenizer = AutoTokenizer.from_pretrained("hyesunyun/update-summarization-bart-large-longformer")

# model = AutoModelForSeq2SeqLM.from_pretrained(my_model).to(device)

time: 222 µs (started: 2024-04-27 08:55:33 +00:00)


In [ ]:
# torch.cuda.empty_cache()

time: 279 µs (started: 2024-04-27 08:55:33 +00:00)


In [ ]:
!nvidia-smi

Sat Apr 27 08:55:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              49W / 400W |   3735MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
#########
# rerun
# important variables
# torch.cuda.empty_cache()

my_num_beams = 3
max_token_length = 900 # 1024
portion_summary = int(max_token_length/3)
summary_max_length = portion_summary
recurisve_batch_size = 1
overlap_percent = 20
print_var = False
##


# error
# RuntimeError: CUDA error: device-side assert triggered
# CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
# For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
# Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# bart
scores_fineTuned, decoded_summaries_lst = evaluate_summaries(dataset_med["test"], metrics , model, tokenizer, column_text="article", column_summary="abstract", batch_size=recurisve_batch_size)


In [ ]:
# model_index = "distilbart-pubmed-20k"
# my_model = "Chung-Fan/distilbart-pubmed-20k"
# og_model = "philschmid/distilbart-cnn-12-6-samsum"

tokenizer = AutoTokenizer.from_pretrained(my_model)
long_tokenizer = AutoTokenizer.from_pretrained(long_tokenizer_model)

model = AutoModelForSeq2SeqLM.from_pretrained(my_model).to(device)

# distilbart
scores_fineTuned, decoded_summaries_lst = evaluate_summaries(dataset_med["test"], metrics , model, tokenizer, column_text="article", column_summary="abstract", batch_size=recurisve_batch_size)


In [ ]:
# pegasus

scores_fineTuned, decoded_summaries_lst = evaluate_summaries(dataset_med["test"], metrics , model, tokenizer, column_text="article", column_summary="abstract", batch_size=recurisve_batch_size)



  0%|          | 0/100 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (1628 > 1024). Running this sequence through the model will result in indexing errors

  1%|          | 1/100 [01:08<1:52:35, 68.23s/it]

i:  0
-------- 
 decoded summary quality check:  ["background: anxiety disorders are associated with neurochemical changes that occur during the early premotor stages of parkinson's disease ( pd ) related degeneration, such as nigrostriatal dopamine depletion and cell loss within serotonergic and noradrenaline brainstem nuclei ( raphe nuclei and locus coeruleus ) which provide massive inputs to corticolimbic regions.results:seven pd patients with anxiety and three pd patients without anxiety were included in this study. self - reported hads was used to assess anxiety in pd and has been previously shown to be a useful measure of clinical anxiety in pd.results:p 0.001 and p 0.001 were statistically significant ( p 0.001 and p 0.001 ) for anxiety and cognition in pd patients with and without anxiety.pda+ and pda+ were statistically significant ( p 0.001 and p 0.001 ) for anxiety and cognition in pd patients with and without anxiety.conclusion:anxiety and depression are often related and c


  2%|▏         | 2/100 [01:34<1:11:24, 43.72s/it]

i:  1



  3%|▎         | 3/100 [01:45<46:31, 28.77s/it]  

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


time: 1h 15min 3s (started: 2024-04-27 08:55:43 +00:00)


In [ ]:
!nvidia-smi

Sat Apr 27 10:10:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0              68W / 400W |   9431MiB / 40960MiB |     51%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
rouge_dict_fineTuned = dict((rn, scores_fineTuned['rouge'][rn].mid.fmeasure) for rn in rouge_names)
bertscore_dict_fineTuned = dict((rn, sum(scores_fineTuned['bert_score'][rn])/len(scores_fineTuned['bert_score'][rn])) for rn in bertscore_names)


time: 559 µs (started: 2024-04-27 10:10:46 +00:00)


In [ ]:
decoded_summaries_lst = list(itertools.chain(*decoded_summaries_lst))
sum_df = pd.read_csv('/content/drive/My Drive/pubmed/recursive_abstracts_4096.csv')
sum_df[model_index+'_overlap' + str(overlap_percent)] = decoded_summaries_lst
sum_df.to_csv('/content/drive/My Drive/pubmed/recursive_abstracts_4096.csv', index=False)
sum_df.head()

,bart-pubmed-20k,bart-pubmed-20k_fluency,bart-pubmed-20k_overlap20,distilbart-pubmed-20k_overlap20,pegasus-pubmed-20k_overlap20
0,background : anxiety affects quality of life i...,"[4, 5, 4, 5, 5]",background : anxiety affects quality of life i...,background : anxiety is the most prominent and...,background: anxiety disorders are associated w...
1,small non - coding rna ( mirnas ) are a class ...,"[5, 5, 4, 5, 5]",small non - coding rna ( mirnas ) are a class ...,small non - coding small rnas ( mirnas ) regul...,background: small non - coding rnas ( mirnas )...
2,background : congenital adrenal hyperplasia ( ...,"[4, 4, 4, 4, 4]",background : congenital adrenal hyperplasia ( ...,background : congenital adrenal hyperplasia ( ...,background : congenital adrenal hyperplasia ( ...
3,background : the family is the cornerstone of ...,"[4, 4, 4, 4, 4]",background : the family is the cornerstone of ...,background : the family is the cornerstone of ...,background:the purpose of this study was to in...
4,objective : the aim of this study was to exami...,"[4, 5, 4, 5, 5]",objective : the aim of this study was to exami...,background : the purpose of nursing care is to...,background:the purpose of the present study is...


time: 462 ms (started: 2024-04-27 10:10:46 +00:00)


In [ ]:
# overlap 20 rouge
pd.DataFrame.from_records(rouge_dict_fineTuned, index=[model_index])

,rouge1,rouge2,rougeL,rougeLsum
pegasus-pubmed-20k,0.300317,0.096325,0.186334,0.253576


time: 11.8 ms (started: 2024-04-27 10:10:47 +00:00)


In [ ]:
# overlap 20 bertscore
pd.DataFrame.from_records(bertscore_dict_fineTuned, index=[model_index])


,f1,precision,recall
pegasus-pubmed-20k,0.817421,0.81076,0.825656


time: 11.2 ms (started: 2024-04-27 10:10:47 +00:00)


In [ ]:
# overlap 20 rouge
pd.DataFrame.from_records(rouge_dict_fineTuned, index=[model_index])


,rouge1,rouge2,rougeL,rougeLsum
distilbart-pubmed-20k,0.394681,0.143522,0.218533,0.334361


In [ ]:
# overlap 20 bertscore
pd.DataFrame.from_records(bertscore_dict_fineTuned, index=[model_index])


,f1,precision,recall
distilbart-pubmed-20k,0.847839,0.84052,0.855609


# GPT fluency evaluation

Exploring GPT fluency evaluation with recursively generated summary samples


In [ ]:
# use the same code as the fluency evaluation notebook, which are removed here for redundancy

In [ ]:
# store fluency score
sum_df[model_index+'_overlap'+overlap_percent+'_fluency'] = fluency_scores
sum_df.to_csv('/content/drive/My Drive/pubmed/recursive_abstracts_4096.csv', index=False)

# Experiment overlap percentage

In [ ]:
# overlap 10 rouge
pd.DataFrame.from_records(rouge_dict_fineTuned, index=[model_index])

,rouge1,rouge2,rougeL,rougeLsum
bart-pubmed-20k,0.400389,0.158904,0.228294,0.342141


In [ ]:
# overlap 10 bertscore
pd.DataFrame.from_records(bertscore_dict_fineTuned, index=[model_index])


,f1,precision,recall
bart-pubmed-20k,0.850731,0.843112,0.858975


In [ ]:
# overlap 20 rouge
pd.DataFrame.from_records(rouge_dict_fineTuned, index=[model_index])


,rouge1,rouge2,rougeL,rougeLsum
bart-pubmed-20k,0.402936,0.153651,0.227064,0.340208


In [ ]:
# overlap 20 bertscore
pd.DataFrame.from_records(bertscore_dict_fineTuned, index=[model_index])


,f1,precision,recall
bart-pubmed-20k,0.850841,0.843547,0.858595


In [ ]:
torch.cuda.empty_cache()  # Free GPU Memory to test on original model.

model_ckpt = og_model
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

scores_original, decoded_summaries_lst_og = evaluate_summaries(dataset_med["test"], metrics, model, tokenizer, column_text="article", column_summary="abstract", batch_size=recurisve_batch_size)
rouge_dict_original = dict((rn, scores_original['rouge'][rn].mid.fmeasure) for rn in rouge_names)
bertscore_dict_original = dict((rn, sum(scores_original['bert_score'][rn])/len(scores_original['bert_score'][rn])) for rn in bertscore_names)


In [ ]:
pd.DataFrame.from_records(rouge_dict_original, index=[f"model without finetuning"])

In [ ]:
pd.DataFrame.from_records(bertscore_dict_original, index=[f"model without finetuning"])

In [ ]:
decoded_summaries_lst = list(itertools.chain(*decoded_summaries_lst))
sum_df = pd.read_csv('/content/drive/My Drive/pubmed/recursive_abstracts_4096.csv')
sum_df[model_index] = decoded_summaries_lst
sum_df.to_csv('/content/drive/My Drive/pubmed/recursive_abstracts_4096.csv', index=False)

# pd.DataFrame.from_records(rouge_dict_fineTuned, index=[model_index])
# pd.DataFrame.from_records(bertscore_dict_fineTuned, index=[model_index])

In [ ]:
pd.DataFrame.from_records(rouge_dict_fineTuned, index=[model_index])

In [ ]:
pd.DataFrame.from_records(bertscore_dict_fineTuned, index=[model_index])

In [ ]:
torch.cuda.empty_cache()  # Free GPU Memory to test on original model.

In [ ]:
model_ckpt = og_model
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

scores_original = evaluate_summaries(dataset_med["test"], metrics, model, tokenizer, column_text="article", column_summary="abstract", batch_size=recurisve_batch_size)
rouge_dict_original = dict((rn, scores_original['rouge'][rn].mid.fmeasure) for rn in rouge_names)
bertscore_dict_original = dict((rn, sum(scores_original['bert_score'][rn])/len(scores_original['bert_score'][rn])) for rn in bertscore_names)

# pd.DataFrame.from_records(rouge_dict_original, index=[f"model without finetuning"])
# pd.DataFrame.from_records(bertscore_dict_original, index=[f"model without finetuning"])

In [ ]:
pd.DataFrame.from_records(rouge_dict_original, index=[f"model without finetuning"])

In [ ]:
pd.DataFrame.from_records(bertscore_dict_original, index=[f"model without finetuning"])

In [ ]:
pd.DataFrame.from_records(rouge_dict_fineTuned, index=[model_index])

,rouge1,rouge2,rougeL,rougeLsum
bart-pubmed-20k,0.430043,0.183356,0.245381,0.37404


In [ ]:
pd.DataFrame.from_records(bertscore_dict_fineTuned, index=[model_index])

,f1,precision,recall
bart-pubmed-20k,0.855604,0.851829,0.860202


In [ ]:

for metric in metrics:
  metric.add_batch(predictions=[a], references=[b])  # add this batch to the metric
scores = dict()
for metric in metrics:
      if str(metric.name) == 'bert_score':
        scores['bert_score'] = metric.compute(lang="en") # Calculate final metric score
      else:
        scores[str(metric.name)] = metric.compute() # Calculate final metric score


In [ ]:
rouge_dict_fineTuned = dict((rn, scores['rouge'][rn].mid.fmeasure) for rn in rouge_names)
bertscore_dict_fineTuned = dict((rn, sum(scores['bert_score'][rn])/len(scores['bert_score'][rn])) for rn in bertscore_names)


In [ ]:
bertscore_dict_fineTuned

{'precision': 0.8809269666671753,
 'recall': 0.8615342378616333,
 'f1': 0.8711226582527161}

In [ ]:
pd.DataFrame.from_records(rouge_dict_fineTuned, index=[model_index])

,rouge1,rouge2,rougeL,rougeLsum
bart-pubmed-20k,0.56262,0.235977,0.331407,0.512524
